<a href="https://colab.research.google.com/github/asifhasan24/codetest/blob/main/Prediction_of_Fetal_Brain_Gestational_Age_Using_Multi_Head_Attention_with_Xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!pwd
!pip install tensorflow-addons
!kaggle datasets download -d asifhasan24/fetal-brain
!unzip /content/fetal-brain.zip

In [ ]:
import random
import cv2
from sklearn.model_selection import train_test_split
import os
import numpy as np
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, MultiHeadAttention
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.optimizers import RMSprop
from sklearn.metrics import mean_absolute_error
import tensorflow as tf

# Load the dataset
image_folder_path = "/content/images"
img_size = 75

# Define a function to apply unsharp masking to an image
def enhance_image(image):
    # Apply Unsharp Masking
    gaussian_blur = cv2.GaussianBlur(image, (0, 0), 3)
    unsharp_image = cv2.addWeighted(image, 1.5, gaussian_blur, -0.8, 0)

    return unsharp_image

# Create a dictionary that maps each image file name to its label
label_dict = {}
for subdirectory in os.listdir(image_folder_path):
    if os.path.isfile(os.path.join(image_folder_path, subdirectory)):
        continue
    for image_file in os.listdir(os.path.join(image_folder_path, subdirectory)):
        if image_file.endswith(".jpg") or image_file.endswith(".jpeg") or image_file.endswith(".png"):
            label_dict[os.path.join(subdirectory, image_file)] = int(subdirectory)

# Shuffle the list of images
img_list = list(label_dict.keys())
random.shuffle(img_list)
print(len(img_list))

# Split the list of images into training, validation, and testing sets
train_images, test_image_list = train_test_split(img_list, test_size=0.3)  # 70% for training
val_image_list, test_image_list = train_test_split(test_image_list, test_size=0.67)  # 10% for validation, 20% for testing

print(len(train_images))
print(len(val_image_list))
print(len(test_image_list))

# Use the new lists to create training, validation, and test sets
train_img, val_img, test_img = [], [], []
train_labels, val_labels, test_labels = [], [], []

for image_file in train_images:
    if image_file in label_dict:
        image = cv2.imread(os.path.join(image_folder_path, image_file))
        image = image / 255.0
        enhanced_image = enhance_image(image)
        train_img.append(cv2.resize(enhanced_image, (img_size, img_size)))
        train_labels.append(label_dict[image_file])
    else:
        print(f"Label not found for image: {image_file}")

for image_file in val_image_list:
    if image_file in label_dict:
        image = cv2.imread(os.path.join(image_folder_path, image_file))
        image = image / 255.0
        enhanced_image = enhance_image(image)
        val_img.append(cv2.resize(enhanced_image, (img_size, img_size)))
        val_labels.append(label_dict[image_file])
    else:
        print(f"Label not found for image: {image_file}")

for image_file in test_image_list:
    if image_file in label_dict:
        image = cv2.imread(os.path.join(image_folder_path, image_file))
        image = image / 255.0
        enhanced_image = enhance_image(image)
        test_img.append(cv2.resize(enhanced_image, (img_size, img_size)))
        test_labels.append(label_dict[image_file])
    else:
        print(f"Label not found for image: {image_file}")

train_img = np.array(train_img)
train_labels = np.array(train_labels)
val_img = np.array(val_img)
val_labels = np.array(val_labels)
test_img = np.array(test_img)
test_labels = np.array(test_labels)

train_img = preprocess_input(train_img)
val_img = preprocess_input(val_img)
test_img = preprocess_input(test_img)

# Load the Xception model pre-trained on ImageNet data
base_model = Xception(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Reshape x to have sequence length dimension
x = tf.expand_dims(x, axis=1)  # Shape: (None, 1, 2048)

# Add multi-head attention layer
num_heads = 8
key_dim = 64
value_dim = 64
attention_output, attention_scores = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim, value_dim=value_dim)(x, x, return_attention_scores=True)

# Flatten the attention output tensor
attention_output = tf.keras.layers.Flatten()(attention_output)

# Add a fully-connected layer with 512 hidden units and relu activation
x = Dense(512, activation='relu')(attention_output)

# Add the output layer with one neuron for regression
output_layer = Dense(1)(x)
model = Model(inputs=base_model.input, outputs=output_layer)
model.compile(loss='mean_squared_error', optimizer=RMSprop(lr=0.001), metrics=['mae'])

# Train the model
model.fit(train_img, train_labels, epochs=100, batch_size=16, validation_data=(val_img, val_labels))

# Evaluate on the test set
y_pred = model.predict(test_img)
r2 = r2_score(test_labels, y_pred)
mae = mean_absolute_error(test_labels, y_pred)
mse = mean_squared_error(test_labels, y_pred)
rmse = np.sqrt(mse)
print("Test Set - R-squared score: {:.3f}".format(r2))
print("Test Set - MAE: {:.3f}".format(mae))
print("Test Set - MSE: {:.3f}".format(mse))
print("Test Set - RMSE: {:.3f}".format(rmse))

# Evaluate on the validation set
val_pred = model.predict(val_img)
val_r2 = r2_score(val_labels, val_pred)
val_mae = mean_absolute_error(val_labels, val_pred)
val_mse = mean_squared_error(val_labels, val_pred)
val_rmse = np.sqrt(val_mse)
print("Validation Set - R-squared score: {:.3f}".format(val_r2))
print("Validation Set - MAE: {:.3f}".format(val_mae))
print("Validation Set - MSE: {:.3f}".format(val_mse))
print("Validation Set - RMSE: {:.3f}".format(val_rmse))


In [ ]:
import random
import cv2
from sklearn.model_selection import train_test_split
import os
import numpy as np
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, MultiHeadAttention
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.optimizers import RMSprop
from sklearn.metrics import mean_absolute_error
import pandas as pd
import tensorflow as tf

# Load the dataset
image_folder_path = "/content/images"
img_size = 75

# Define a function to apply unsharp masking to an image
def enhance_image(image):
    # Apply Unsharp Masking
    gaussian_blur = cv2.GaussianBlur(image, (0, 0), 3)
    unsharp_image = cv2.addWeighted(image, 1.5, gaussian_blur, -0.8, 0)

    return unsharp_image

# Create a dictionary that maps each image file name to its label
label_dict = {}
for subdirectory in os.listdir(image_folder_path):
    if os.path.isfile(os.path.join(image_folder_path, subdirectory)):
        continue
    for image_file in os.listdir(os.path.join(image_folder_path, subdirectory)):
        if image_file.endswith(".jpg") or image_file.endswith(".jpeg") or image_file.endswith(".png"):
            label_dict[os.path.join(subdirectory, image_file)] = int(subdirectory)

# Shuffle the list of images
img_list = list(label_dict.keys())
random.shuffle(img_list)

excel_file_path = '/content/images/labels.xlsx'
df = pd.read_excel(excel_file_path)

df['cor_series'] = df['cor_series'].astype(str)
df['cor_series'] = df['cor_series'].apply(lambda x: x.zfill(4))
cor_series_column = df['cor_series']

# Filter the image list based on cor_series values
filtered_img_list = [image_file for image_file in img_list if any((f"-{cor_series_value}-" in image_file) for cor_series_value in cor_series_column)]
print(len(filtered_img_list))

# Split the list of images into training, validation, and testing sets
train_val_images, test_image_list = train_test_split(filtered_img_list, test_size=0.2)
train_image_list, val_image_list = train_test_split(train_val_images, test_size=0.125)  # 10% for validation

print(len(train_image_list))
print(len(val_image_list))
print(len(test_image_list))

# Create image and label arrays for training, validation, and testing
train_img, val_img, test_img = [], [], []
train_labels, val_labels, test_labels = [], [], []

# Load and preprocess images for training set
for image_file in train_image_list:
    if image_file in label_dict:
        image = cv2.imread(os.path.join(image_folder_path, image_file))
        image = image / 255.0
        enhanced_image = enhance_image(image)
        train_img.append(cv2.resize(enhanced_image, (img_size, img_size)))
        train_labels.append(label_dict[image_file])
    else:
        print(f"Label not found for image: {image_file}")

# Load and preprocess images for validation set
for image_file in val_image_list:
    if image_file in label_dict:
        image = cv2.imread(os.path.join(image_folder_path, image_file))
        image = image / 255.0
        enhanced_image = enhance_image(image)
        val_img.append(cv2.resize(enhanced_image, (img_size, img_size)))
        val_labels.append(label_dict[image_file])
    else:
        print(f"Label not found for image: {image_file}")

# Load and preprocess images for test set
for image_file in test_image_list:
    if image_file in label_dict:
        image = cv2.imread(os.path.join(image_folder_path, image_file))
        image = image / 255.0
        enhanced_image = enhance_image(image)
        test_img.append(cv2.resize(enhanced_image, (img_size, img_size)))
        test_labels.append(label_dict[image_file])
    else:
        print(f"Label not found for image: {image_file}")

# Convert lists to numpy arrays
train_img = np.array(train_img)
train_labels = np.array(train_labels)
val_img = np.array(val_img)
val_labels = np.array(val_labels)
test_img = np.array(test_img)
test_labels = np.array(test_labels)

# Preprocess input images
train_img = preprocess_input(train_img)
val_img = preprocess_input(val_img)
test_img = preprocess_input(test_img)

# Load the Xception model pre-trained on ImageNet data
base_model = Xception(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Reshape x to have sequence length dimension
x = tf.expand_dims(x, axis=1)  # Shape: (None, 1, 2048)

# Add multi-head attention layer
num_heads = 8
key_dim = 64
value_dim = 64
attention_output, attention_scores = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim, value_dim=value_dim)(x, x, return_attention_scores=True)

# Flatten the attention output tensor
attention_output = tf.keras.layers.Flatten()(attention_output)

# Add a fully-connected layer with 512 hidden units and relu activation
x = Dense(512, activation='relu')(attention_output)

# Add the output layer with one neuron for regression
output_layer = Dense(1)(x)
model = Model(inputs=base_model.input, outputs=output_layer)
model.compile(loss='mean_squared_error', optimizer=RMSprop(lr=0.001), metrics=['mae'])

# Train the model
model.fit(train_img, train_labels, epochs=100, batch_size=16, validation_data=(val_img, val_labels))

# Evaluate on the test set
y_pred = model.predict(test_img)
r2 = r2_score(test_labels, y_pred)
mae = mean_absolute_error(test_labels, y_pred)
mse = mean_squared_error(test_labels, y_pred)
rmse = np.sqrt(mse)
print("Test Set - R-squared score: {:.3f}".format(r2))
print("Test Set - MAE: {:.3f}".format(mae))
print("Test Set - MSE: {:.3f}".format(mse))
print("Test Set - RMSE: {:.3f}".format(rmse))

# Evaluate on the validation set
val_pred = model.predict(val_img)
val_r2 = r2_score(val_labels, val_pred)
val_mae = mean_absolute_error(val_labels, val_pred)
val_mse = mean_squared_error(val_labels, val_pred)
val_rmse = np.sqrt(val_mse)
print("Validation Set - R-squared score: {:.3f}".format(val_r2))
print("Validation Set - MAE: {:.3f}".format(val_mae))
print("Validation Set - MSE: {:.3f}".format(val_mse))
print("Validation Set - RMSE: {:.3f}".format(val_rmse))

# Save the model
model.save('/content/modelcor.h5')


In [ ]:
import matplotlib.pyplot as plt

# Plotting the actual vs predicted values for the test set
plt.figure(figsize=(10, 6))
plt.scatter(test_labels, y_pred, color='blue', alpha=0.5, label='Actual vs Predicted')
plt.title('Actual vs Predicted Values (Test Set)')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.show()
